In [1]:
import pandas as pd
from keybert import KeyBERT
from collections import Counter
from tqdm import tqdm

In [13]:
df = pd.read_csv('data/충청북도_전처리_2.csv')

In [14]:
df.head(3)

,시도,시군구,code,token
0,충청북도,괴산군,43760,광 저수지 은행나무 길 국내 가을 광 저수지 은행나무 길 광 저수지 은행나무 절정 ...
1,충청북도,괴산군,43760,군별 팔경 목록 여행명소 명소 군별 경 목록 정리 상당산성 둘러싸이다 구름 무심천 ...
2,충청북도,괴산군,43760,이야기 결과 후기 기간 수목 박일 여정 중부 지역 특이 사항 은퇴 부부 차 역사 인...


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4023 entries, 0 to 4022
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      4023 non-null   object
 1   시군구     4023 non-null   object
 2   code    4023 non-null   int64 
 3   token   4023 non-null   object
dtypes: int64(1), object(3)
memory usage: 125.8+ KB


In [16]:
# KeyBERT를 활용한 키워드 추출 함수
def keybert_extract_keywords(text, top_n=20):
    # KeyBERT 초기화
    model = KeyBERT(model="all-MiniLM-L6-v2") 
    # 각 문서별 상위 top_n개 키워드와 스코어 추출
    keyword_score = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), top_n=top_n)
    # 추출된 키워드를 공백을 이용해 문자열로 결합
    keywords = ' '.join(keyword for keyword, score in keyword_score)
    return keywords

In [17]:
# 빈도수 기반 키워드 추출 함수
def extract_top_keywords(text, top_n=10):
    words = text.split()
    word_counts = Counter(words)
    keywords = ' '.join(word for word, _ in word_counts.most_common(top_n))
    return keywords

In [18]:
tqdm.pandas()

# 함수 실행
df['keyword1'] = df['token'].progress_apply(keybert_extract_keywords)
df['keyword2'] = df['token'].progress_apply(extract_top_keywords)

100%|████████████████████████████████████████████████████████████████████████████| 4023/4023 [00:01<00:00, 3682.53it/s]


In [19]:
# 키워드 병합 및 중복 제거
df['keyword'] = df.apply(lambda row: ' '.join(set((row['keyword1'] + ' ' + row['keyword2']).split())), axis=1)

In [20]:
df = df[['시도', '시군구', 'code', 'token', 'keyword']]

In [21]:
df.head(3)

,시도,시군구,code,token,keyword
0,충청북도,괴산군,43760,광 저수지 은행나무 길 국내 가을 광 저수지 은행나무 길 광 저수지 은행나무 절정 ...,절정 산책 생산 방문객 문광 캠핑장 저수지 문광면 가을 줄이다 명상 많다 농작물 단...
1,충청북도,괴산군,43760,군별 팔경 목록 여행명소 명소 군별 경 목록 정리 상당산성 둘러싸이다 구름 무심천 ...,암 문장 노을 저녁 무심천 천팔경 달 구성지다 명소 삼년산성 천정 속리산 소리 구름...
2,충청북도,괴산군,43760,이야기 결과 후기 기간 수목 박일 여정 중부 지역 특이 사항 은퇴 부부 차 역사 인...,청동기 이화숙 금속활자 마을 전시관 세종대왕 전시 도착 걸리다 삼년산성 불조직지심체...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4023 entries, 0 to 4022
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시도       4023 non-null   object
 1   시군구      4023 non-null   object
 2   code     4023 non-null   int64 
 3   token    4023 non-null   object
 4   keyword  4023 non-null   object
dtypes: int64(1), object(4)
memory usage: 157.3+ KB


In [23]:
df.to_csv('data/충청북도_키워드.csv', encoding='utf-8-sig', index=False)